# Advent of Code 2018

This solution (Jupyter notebook; python3.7) by kannix68, @ 2020-12 (2 years late).  \
Using anaconda distro, conda v4.9.2. installation on MacOS v10.14.6 "Mojave".

## Generic AoC code

In [ ]:
import sys
print("Python version:", sys.version)
print("Version info:", sys.version_info)

In [ ]:
DEBUG_FLAG = 0
VERBOSE_LEVEL = 1

In [ ]:
# Code functions
# TODO: import and use lib/aochelper instead

def assert_msg(msg, assertion):
  """Assert boolean condition with message, ok=message printed, otherwise assertion-fail."""
  assert assertion, "ERROR on assert: {}".format(msg)
  print("assert-OK: {}".format(msg))

def expect_msg(msg, expected, inputs):
  assert_msg(msg.format(inputs, expected), expected == solve(inputs))

def log_debug(*args,**kwargs):
  """Print message only if DEBUG_FLAG > 0."""
  if DEBUG_FLAG > 0:
    print('D: ', end='')
    print(*args,**kwargs)

def log_info(*args,**kwargs):
  """Print message only if VERBOSE_LEVEL > 0."""
  if VERBOSE_LEVEL > 0:
    print('I: ', end='')
    print(*args,**kwargs)

def read_file_to_str(filename):
  """Read a file's content into one string."""
  with open(filename, 'r') as inputfile:
    data = inputfile.read()
  return data

def read_file_to_list(filename):
  """Read a file's content into a list of strings (per line), each ending whitespace stripped."""
  with open(filename, 'r') as inputfile:
    lines_list = inputfile.readlines()
  #lines_list = [line.rstrip('\n') for line in lines_list] # via list comprehension
  lines_list = list(map(lambda it: it.rstrip(), lines_list)) # via map
  return lines_list

def lrange(*args,**kwargs):
  return list(range(*args,**kwargs))

def lmap(*args,**kwargs):
  return list(map(*args,**kwargs))

def lfilter(*args,**kwargs):
  return list(filter(*args,**kwargs))


## Problem domain code

### Day 1: Chronal Calibration

In [ ]:
print("Day 1 a")

In [ ]:
test = """
+1, -2, +3, +1
""".strip().split(', ')
tests = lmap(int, test)
assert_msg("test 1", 3 == sum(tests))

In [ ]:
ins = read_file_to_list('./day01/day01.in')
print("Day 1 a solution:", sum(lmap(int, ins)))

In [ ]:
print("Day 1 b", "TODO here, but already solved 2018")

In [ ]:
import itertools

In [ ]:
# A list and list.append(frequency are resource hog tools to keep track of seen entries),
#  using dict instead.
def solve01b(l):
  iter = 0
  freq = 0
  freqs = {freq:True}
  for freq_inc in itertools.cycle(l):
    iter += 1
    freq += freq_inc
    if (len(freqs)%100_000 == 0):
      log_info(f"iter={iter}, freq={freq} num-frequencies={len(freqs)}")
    if freq in freqs:
      log_info(f"frequency {freq} used 2nd time, iteration={iter}, num-frequencies={len(freqs)}")
      return freq
    elif iter > 10_000_000:
      raise Exception("fail")
    else:
      freqs[freq] = True


In [ ]:
solve01b(tests)
iins = lmap(int, ins)
log_debug(len(iins), iins)
result = solve01b(iins)
print("Day 1 b result:", result)

### Day 2: Inventory Management System

In [ ]:
print("Day 2", "TODO here, but already solved 2018")

### Day 3: No Matter How You Slice It

In [ ]:
from collections import defaultdict

class C2d_space_inc:
  
  def __init__(self):
    self.spc = defaultdict(int)
    self.clms = {}
  
  def set_xy(self, x, y):
    self.spc[tuple([x,y])] += 1
  
  def set_range(self, x, y, rgx, rgy, id=None):
    for px in range(x, x+rgx):
      for py in range(y, y+rgy):
        self.spc[tuple([px,py])] += 1
    if id is not None:
      self.clms[id] = [x, y, rgx, rgy]
      #log_info(f"create claim {id} => {self.claims[id] }")
  
  def get_range(self, x, y, rgx, rgy):
    outspc = {}
    for px in range(x, x+rgx):
      for py in range(y, y+rgy):
        outspc[tuple([px,py])] = self.spc[tuple([px,py])]
    return outspc

  def cols(self):
    return sorted(set(map(lambda it: it[0], self.spc.keys())))

  def rows(self):
    return sorted(set(map(lambda it: it[1], self.spc.keys())))
  
  def values(self):
    return self.spc.values()

  def claims(self):
    return self.clms

  def get_pp_repr(self):
    return "[c2d_space_inc]: " + str(self.spc)

  def pp(self):
    print(self.get_pp_repr())

  def get_show_repr(self):
    rows = self.rows()
    cols = self.cols()
    rowstr = ''
    for y in range(0, max(rows)+1): #range(min(rows), max(rows)+1):
      colstr = ''
      for x in range(0, max(cols)+1):
        colstr += str(self.spc[tuple([x,y])])
      rowstr += colstr + "\n"
    return rowstr

  def show(self):
    print(self.get_show_repr())

In [ ]:
# just some testing:
c2d = C2d_space_inc()
c2d.set_xy(1,1)
c2d.set_xy(2,2)
log_debug(c2d.get_pp_repr())
c2d.set_xy(1,1)
c2d.set_xy(1,3)
log_debug(c2d.get_pp_repr())
log_debug("cols:", c2d.cols())
log_debug("rows:", c2d.rows())
log_debug("\n"+c2d.get_show_repr()) #c2d.show()

In [ ]:
def create_space(l):
  import re
  c2d = C2d_space_inc()
  for line in l:
    rx = re.search('^#(\w+)\s+@\s+(\d+),(\d+):\s*(\d+)x(\d+)$', line) #r'^#([^\s]+) @ (d+),(\d+): ((\d+)),((\d+))$', line)
    log_debug(rx.groups())
    id, sx, sy, srgx, srgy = rx.groups()
    x, y, rgs, rgy = lmap(int, [sx, sy, srgx, srgy])
    #c2d.set_range(x, y, rgs, rgy)
    c2d.set_range(x, y, rgs, rgy, id=id)
  #c2d.show()
  return c2d

In [ ]:
tests = """
#123 @ 3,2: 5x4
""".strip().split("\n")
log_info("tests", tests)

In [ ]:
print("1st test representation:")
create_space(tests).show()

In [ ]:
tests = """
#1 @ 1,3: 4x4
#2 @ 3,1: 4x4
#3 @ 5,5: 2x2
""".strip().split("\n")
log_info("tests", tests)

In [ ]:
print("2nd test representation:")
create_space(tests).show()

In [ ]:
def get_space_overlaps(l):
  c2d = create_space(l)
  return len(lfilter(lambda it: it > 1, c2d.values()))

In [ ]:
assert_msg( "4 test cells that overlap", 4 == get_space_overlaps(tests) )

In [ ]:
ins = read_file_to_list('./day03/day03.in')
result = get_space_overlaps(ins)
print("Day 3 a solution:", result)

In [ ]:
def get_singular_space(l):
  c2d = create_space(l)
  for k, v in c2d.claims().items():
    log_debug(k, v)
    rg = c2d.get_range(v[0], v[1], v[2], v[3])
    log_debug(rg)
    log_debug(rg.values())
    if max(rg.values()) == 1:
      log_info("found id:", k, ", num of 1-cells:", len(rg.values()))
      result = k
  return result

In [ ]:
print("Day 3 b tests result: ", get_singular_space(tests))
print("Day 3 b solution:", get_singular_space(ins))

### Day 4

In [ ]:
# TODO

### Day 5

In [ ]:
# TODO

### Day 6:  Chronal Coordinates

In [ ]:
#TODO

### Day 7: The Sum of Its Parts

In [ ]:
DEBUG_FLAG = 0

In [ ]:
import networkx as nx

In [ ]:
tests = """
Step C must be finished before step A can begin.
Step C must be finished before step F can begin.
Step A must be finished before step B can begin.
Step A must be finished before step D can begin.
Step B must be finished before step E can begin.
Step D must be finished before step E can begin.
Step F must be finished before step E can begin.
""".strip().split("\n")

In [ ]:
def create_graph(l):
  graph = nx.DiGraph()
  for line in l:
    linesplit = line.split(' ')
    srcnd, trgnd = [linesplit[1], linesplit[-3]]
    if not srcnd in graph.nodes:
      log_debug(f"g add node {srcnd}")
    if not trgnd in graph.nodes:
      log_debug(f"g add node {trgnd}")
    graph.add_edge(srcnd, trgnd)
  log_info("graph-edges:", sorted(list(graph.edges)))
  return graph

In [ ]:
# still using networkx graph li, inspiration from user VikeStep:
#  [- 2018 Day 7 Solutions - : adventofcode](https://www.reddit.com/r/adventofcode/comments/a3wmnl/2018_day_7_solutions/)
def solve07a(l):
  graph = create_graph(l)
  
  nodes_lst = list(graph.nodes)
  log_debug("nodes: entry-order", str.join('', nodes_lst))
  nodes_lst = list(nx.topological_sort(graph))
  log_debug("nodes: topo", str.join('', nodes_lst))
  nodes_lst = list(nx.algorithms.dag.lexicographical_topological_sort(graph))
  log_info("nodes: lexico-topo", str.join('', nodes_lst))
  return str.join('', nodes_lst)

In [ ]:
solve07a(tests)

In [ ]:
ins = read_file_to_list('./in/day07.in')
solve07a(ins)

In [ ]:
print("Day 07 b")

In [ ]:
#TODO

In [ ]:
### Day 14: Chocolate Charts

In [ ]:
def find_improved_score(target_num):
  elf1 = 0
  elf2 = 1
  recipies = [3, 7]
  #num_new_recipies = 0
  log_info(f"#0: num-recipes={len(recipies)}, recipies={recipies}")
  for i in range(2*target_num):
    new_recipy = recipies[elf1] +  recipies[elf2]
    #found_recipies.add(new_recipy)
    digits = lmap(int, str(new_recipy))
    #num_new_recipies += len(digits)
    for d in digits:
      recipies.append(d)
    elf1 = (elf1+1+recipies[elf1]) % len(recipies)
    elf2 = (elf2+1+recipies[elf2]) % len(recipies)
    log_debug(f"#{i+1}: num-recipes={len(recipies)}")
    log_debug(len(digits), digits, recipies, " elf1:", elf1, " elf2:", elf2, len(recipies)) #found_recipies)
    if len(recipies) >= target_num + 10:
      res = str.join('', lmap(str, recipies))[target_num:target_num+10] # 0124515891
      log_info("found:", res)
      return res


In [ ]:
print( find_improved_score(9) )
print( find_improved_score(5) )
print( find_improved_score(18) )
print( find_improved_score(2018) )


In [ ]:
ins = 846021 # << insert personal input here
print( "Day 14 a solution:", find_improved_score(ins) )

In [ ]:
print("Day 14 b")

def find_num_recips_before(target_num):
  target_str = str(target_num)
  elf1 = 0
  elf2 = 1
  recipies = [3, 7]
  #num_new_recipies = 0
  log_info(f"#0: num-recipes={len(recipies)}, recipies={recipies}")
  for i in range(1000*int(target_num)):
    new_recipy = recipies[elf1] +  recipies[elf2]
    #found_recipies.add(new_recipy)
    digits = lmap(int, str(new_recipy))
    #num_new_recipies += len(digits)
    for d in digits:
      recipies.append(d)
    elf1 = (elf1+1+recipies[elf1]) % len(recipies)
    elf2 = (elf2+1+recipies[elf2]) % len(recipies)
    #log_debug(f"#{i+1}: num-recipes={len(recipies)}")
    #log_debug(len(digits), digits, recipies, " elf1:", elf1, " elf2:", elf2, found_recipies)
    if i % 1_000_000 == 0:
      #True
      log_info("calculating, iter:", i)
    recips_end_str = str.join('', lmap(str, recipies[-12:]))
    if target_str in recips_end_str:
      offset = 0
      if not recips_end_str.endswith(target_str):
        recips_end_str = recips_end_str[0:-1]
        offset = 1
      assert( recips_end_str.endswith(target_str) )
      #recips_str = str.join('', lmap(str, recipies))
      #res = recips_str.index(target_str)
      log_info(len(recipies)-len(target_str)) #" from {recips_str}")
      res = len(recipies) - len(target_str) - offset
      log_info(f"target-num={target_str}, found: idx={res} @iter={i}") #" from {recips_str}")
      return res
  raise Exception("not terminated with find-criterium")

In [ ]:
find_num_recips_before(51589)

In [ ]:
find_num_recips_before('01245')
find_num_recips_before('92510')
find_num_recips_before('59414')

In [ ]:
PERFORM_RESOURCE_HOGS = False
if PERFORM_RESOURCE_HOGS:
  find_num_recips_before(ins)